<a href="https://colab.research.google.com/github/shu-bee/Pytorch_tutorial/blob/main/neural_network_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
params = list(net.parameters())
print(params[0])
print(len(params))
print(params[0].size())

Parameter containing:
tensor([[[[-0.0305, -0.1144,  0.1985, -0.0225,  0.0103],
          [ 0.1752, -0.1226, -0.1910,  0.1956, -0.0213],
          [ 0.1349, -0.1718, -0.1893,  0.1885,  0.0675],
          [-0.1159,  0.0475,  0.0047, -0.1345, -0.0211],
          [-0.0328, -0.0231,  0.0477, -0.1164,  0.1399]]],


        [[[ 0.0914, -0.0177, -0.1167,  0.0750,  0.1361],
          [-0.0639,  0.0815,  0.1863, -0.0945, -0.0779],
          [ 0.1772,  0.0040, -0.0498,  0.1344, -0.1298],
          [ 0.1839,  0.0424, -0.0454, -0.0510,  0.1141],
          [ 0.1123, -0.1767,  0.1846,  0.1612, -0.0925]]],


        [[[-0.1688,  0.0753, -0.1160, -0.0544, -0.1728],
          [ 0.1201,  0.1815, -0.0223, -0.0487, -0.1719],
          [-0.1817,  0.1881,  0.1771, -0.0523, -0.0298],
          [-0.0502, -0.0994, -0.1961, -0.1250,  0.1226],
          [-0.0519,  0.1563, -0.1176, -0.1556, -0.0508]]],


        [[[-0.1146, -0.0247,  0.0994, -0.0856, -0.0749],
          [-0.0819,  0.1218, -0.0241, -0.1161,  0.1970

In [4]:
input = torch.randn(1,1,32,32,requires_grad=True)
out=net(input)
print(out)

tensor([[-0.0286,  0.1121,  0.0136, -0.0595, -0.0150,  0.0422,  0.0526,  0.0399,
          0.0162,  0.0329]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))
print(input.grad)

tensor([[[[ 8.0381e-05,  4.0336e-06, -4.1977e-05,  ...,  1.0093e-03,
            5.4710e-06,  0.0000e+00],
          [ 2.3802e-04,  2.1642e-04, -2.9906e-04,  ...,  9.7526e-04,
            2.5395e-04,  6.7116e-05],
          [-2.8222e-04, -3.8173e-04,  7.9560e-04,  ..., -1.0684e-03,
            5.5692e-04,  1.0484e-05],
          ...,
          [ 5.4998e-06,  5.4259e-04, -9.3578e-04,  ...,  4.4438e-04,
            2.3118e-04, -6.0682e-04],
          [ 7.5671e-06,  1.9572e-04, -2.6148e-04,  ..., -3.7312e-04,
            3.0550e-04, -5.1511e-06],
          [ 0.0000e+00,  1.2535e-04, -2.2849e-04,  ..., -7.1257e-04,
            3.3837e-04, -3.0602e-04]]]])


output=net(input)
target=torch.randn(10)
target=target.view(1,-1)
criterion=nn.MSELoss()
print(out)
print(target)

loss=criterion(output,target)
print(loss)

In [21]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])

tensor(2.4025, grad_fn=<MseLossBackward>)


In [22]:
net.zero_grad()

print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0013,  0.0117, -0.0030,  0.0092,  0.0138, -0.0136])


In [19]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [20]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.01)

output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()